In [1]:
import tensorflow as tf
import numpy as np
from datetime import datetime
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler

# gradient descent
housing = fetch_california_housing()
m, n = housing["data"].shape
housing_data_plus_bias = np.c_[np.ones([m,1]), housing["data"]]
scaler = StandardScaler()
scaled_housing_data_plus_bias = scaler.fit_transform(housing_data_plus_bias)
n_epochs = 1000
learning_rate = 0.01
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing["target"].reshape(-1,1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n+1,1], -1, 1), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)
init = tf.global_variables_initializer()

# create log directory
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

# create note and write
mse_summary = tf.summary.scalar("MSE", mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

# run session
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch % 10 == 0:
            summary_str = mse_summary.eval()
            step = epoch
            #file_writer.add_summary(summary_str, step)
        sess.run(training_op)
    best_theta = theta.eval()
file_writer.close()
tf.reset_default_graph()

# modularity and sharing variables
graph = tf.Graph()
with graph.as_default():
    def relu(X):
        with tf.name_scope("relu"):
            with tf.variable_scope("MAP", reuse=True):
                threshold = tf.get_variable("threshold")
            w_shape = (int(X.get_shape()[1]), 1)
            w = tf.Variable(tf.random_normal(w_shape), name="weights")
            b = tf.Variable(0.0, name="bias")
            z = tf.add(tf.matmul(X, w), b, name="z")
            return tf.maximum(z, threshold, name="relu")
    n_features = 3
    X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
    with tf.variable_scope("MAP"):
        threshold = tf.get_variable("threshold", shape=(), 
                                    initializer=tf.constant_initializer(0.0))
    relus = [relu(X) for i in range(5)]
    output = tf.add_n(relus, name="output")
file_writer = tf.summary.FileWriter(logdir, graph=graph)
file_writer.flush()